Aaron Chen
aaron.chen@spc.ox.ac.uk

Companies House Scraped Officer/ VIPs Data Collector. 

Collect the scraped company_officer and company_VIP json files. Returns a DataFrame with all information. 

Naming Convention: 
merge_{start month}_{end month}.csv

In [6]:
import networkx as nx
import pandas as pd
import numpy as np
from tqdm import tqdm
import glob
import json
import matplotlib.pyplot as plt
import re
from random import randint
import operator

In [7]:
def mapping(date): 
    year = date.split('-')[0]
    return year

In [8]:
company_id = set([i.split('/')[-2] for i in glob.glob('data_2/*/*')   if 'company_officers' in i ])

In [9]:
empty_DOB = {'month':0, 'year': 0}
empty_address = {'postal_code':'0'}

In [10]:
officer_data = []
for c in tqdm(company_id): 
    info = json.loads(open(f'data_2/{c}/company_officers.json').read())
    for i in info['items']: 
        officers = {}
        try: 
            name = str(i['identification']['registration_number']).upper()
            if len(name) == 7: 
                name = '0' + name
            officers['name'] = name
        except: 
            officers['name']=i.get('name', 'no_name')
        officers['occupation'] = i.get('occupation', 'no_occupation')
        officers['officer_role'] = i.get('officer_role', 'no_role')
        officers['appointed_on']= i.get('appointed_on', 'no_appointed_date')
        officers['country_of_residence'] = i.get('country_of_residence', 'no_country')
        officers['nationality'] = i.get('nationality', 'no_nationality')
        officers['resigned_on'] = i.get('resigned_on', 'still_appointed')
        address = i.get('address', empty_address)
        officers['postal_code'] = address.get('postal_code', '0')
        DOB = i.get('date_of_birth', empty_DOB)
        month = DOB.get('month', 0)
        year = DOB.get('year', 0)
        officers['DOB'] = f'{year}-{month}'
        officers['company_id'] = c
        
        officer_data.append(officers)

officer_data_DF = pd.DataFrame(officer_data)

100%|█████████████████████████████████| 470342/470342 [02:42<00:00, 2895.42it/s]


In [11]:
officer_data_DF.loc[:, ('relation')] = officer_data_DF['officer_role']

In [12]:
# officer_data_DF.loc[:, 'appointed_on'] = officer_data_DF['appointed_on'].map(mapping)
# officer_data_DF.loc[:, 'resigned_on'] = officer_data_DF['resigned_on'].map(mapping)

In [13]:
officer_data_DF['start'] = officer_data_DF['appointed_on']
officer_data_DF['end'] = officer_data_DF['resigned_on']
#officer_data_DF
officer_data_DF['kind'] = officer_data_DF['officer_role']

In [14]:
#officer_data_DF

In [15]:
company_id_vip = set([i.split('/')[-2] for i in glob.glob('data_2/*/*') if 'company_VIP' in i ])

In [16]:
VIP_data = []
for c in tqdm(company_id_vip): 
    data = json.loads(open(f'data_2/{c}/company_VIP.json').read())
    for i in data['items']: 
        vips = {}
        try: 
            vips['name'] = i['identification']['registration_number'].upper()
            if len(name) == 7: 
                name = '0' + name
            officers['name'] = name
        except: 
            try : 
                vips['name'] = i['name_elements']['surname'].upper()+', '+i['name_elements']['forename']
            except : 
                vips['name'] = i.get('name', 'protected')
            
            
        vips['natures_of_control'] = i.get('natures_of_control', 'no_info')
        vips['kind'] = i.get('kind', 'no_info')
        vips['company_id'] = c
        vips['notified_on'] = i.get('notified_on', 'no_info')
        vips['ceased_on'] = i.get('ceased_on', 'active')
        address = i.get('address', empty_address)
        vips['postal_code'] = address.get('postal_code', '0')
        DOB = i.get('date_of_birth', empty_DOB)
        month = DOB.get('month', 0)
        year = DOB.get('year', 0)
        vips['DOB'] = f'{year}-{month}'
        
        
        VIP_data.append(vips)

VIP_DF = pd.DataFrame(VIP_data)

100%|█████████████████████████████████| 448476/448476 [04:15<00:00, 1754.73it/s]


In [17]:
VIP_DF = VIP_DF.explode('natures_of_control')
VIP_DF['relation'] = VIP_DF['natures_of_control']

In [18]:
# VIP_DF.loc[:, 'notified_on'] = VIP_DF['notified_on'].map(mapping)
# VIP_DF.loc[:, 'ceased_on'] = VIP_DF['ceased_on'].map(mapping)
VIP_DF['start'] = VIP_DF['notified_on']
VIP_DF['end'] = VIP_DF['ceased_on']
#VIP_DF

In [19]:
#VIP_DF[VIP_DF['start'] == 'no_info']

In [20]:
vip_df = VIP_DF[['name', 'relation', 'kind', 'company_id', 'start', 'end', 'postal_code', 'DOB']]
vip_df['type'] = ['vip' for i in range(len(vip_df.index))]
#vip_df.tail()

/var/folders/9p/frgccnx948gbc26h57pgdn_00000gn/T/ipykernel_4450/1563142686.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vip_df['type'] = ['vip' for i in range(len(vip_df.index))]


In [21]:
officer_df = officer_data_DF[['name', 'relation', 'kind', 'company_id', 'start', 'end', 'postal_code', 'DOB']]
officer_df.loc[:, ('type')] = ['officer' for i in range(officer_df.shape[0])]
#officer_df.tail()

/var/folders/9p/frgccnx948gbc26h57pgdn_00000gn/T/ipykernel_4450/2544483640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  officer_df.loc[:, ('type')] = ['officer' for i in range(officer_df.shape[0])]


In [22]:
merge = pd.concat([officer_df, vip_df])
merge.reset_index(drop=True, inplace=True)
#merge

In [23]:
# name convention -> 'merge_' + '[1-12]' + '_' + ... 
merge.to_csv('merge_7_12.csv', index = False)

In [24]:
merge

,name,relation,kind,company_id,start,end,postal_code,DOB,type
0,"HENNOCK, Christine",director,director,10968888,2017,still_appointed,RH13 0PS,1960-6,officer
1,"HENNOCK, Martin",director,director,10968888,2017,still_appointed,RH13 0PS,1960-6,officer
2,"HALPIN, Tj",director,director,11912297,2019,still_appointed,TN25 4BF,1995-8,officer
3,"LEWIS, Joan Elizabeth",secretary,secretary,07893589,2011,2021,CB4 0WX,0-0,officer
4,"LEWIS, Michael",director,director,07893589,2011,2021,CB4 0WX,1958-10,officer
...,...,...,...,...,...,...,...,...,...
2873884,"FLETCHER, Martin",right-to-appoint-and-remove-directors,individual-person-with-significant-control,10705191,2017,active,RM12 6RJ,1963-4,vip
2873885,13231271,voting-rights-75-to-100-percent-limited-liabil...,corporate-entity-person-with-significant-control,OC360885,2022,active,NW4 1RL,0-0,vip
2873886,"WRAY, Nigel",voting-rights-75-to-100-percent-limited-liabil...,individual-person-with-significant-control,OC360885,2018,2022,W1G 0PJ,1948-4,vip
2873887,"WRAY, Nigel",right-to-share-surplus-assets-25-to-50-percent...,individual-person-with-significant-control,OC360885,2018,2022,W1G 0PJ,1948-4,vip
